In [4]:
import pandas as pd
import sqlite3

conn = sqlite3.connect('database.sqlite')

query = "SELECT * FROM sqlite_master"

df_schema = pd.read_sql_query(query, conn)

df_schema.tbl_name.unique()

array(['sqlite_sequence', 'Player_Attributes', 'Player', 'Match',
       'League', 'Country', 'Team', 'Team_Attributes'], dtype=object)

In [6]:
query = "SELECT * FROM Player_Attributes"

check_table = pd.read_sql_query(query, conn)
check_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183978 entries, 0 to 183977
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   183978 non-null  int64  
 1   player_fifa_api_id   183978 non-null  int64  
 2   player_api_id        183978 non-null  int64  
 3   date                 183978 non-null  object 
 4   overall_rating       183142 non-null  float64
 5   potential            183142 non-null  float64
 6   preferred_foot       183142 non-null  object 
 7   attacking_work_rate  180748 non-null  object 
 8   defensive_work_rate  183142 non-null  object 
 9   crossing             183142 non-null  float64
 10  finishing            183142 non-null  float64
 11  heading_accuracy     183142 non-null  float64
 12  short_passing        183142 non-null  float64
 13  volleys              181265 non-null  float64
 14  dribbling            183142 non-null  float64
 15  curve            

In [5]:
query = "SELECT COUNT(*) FROM Match"

check_table = pd.read_sql_query(query, conn)
check_table.head()

,COUNT(*)
0,25979


#### 1. Which team scored the most points when playing at home?

In [60]:

query = """
SELECT team_long_name, CAST(SUM(home_team_goal) AS INT) as Home_Goals
FROM Match
LEFT JOIN Team 
ON Match.home_team_api_id = Team.team_api_id
GROUP BY team_long_name;"""

home_goals = pd.read_sql_query(query, conn)
home_goals.sort_values(by = 'Home_Goals', ascending = False, inplace = True)

home_goals.head(1)

,team_long_name,Home_Goals
211,Real Madrid CF,505


##### Answer: Real Madrid CF

<br>

#### 2. Did this team also score the most points when playing away?

In [65]:
query = """
SELECT team_long_name, CAST(SUM(away_team_goal) AS INT) as Away_Goals
FROM Match
LEFT JOIN Team 
ON Match.away_team_api_id = Team.team_api_id
GROUP BY team_long_name;"""

away_goals = pd.read_sql_query(query, conn)
away_goals.sort_values(by = 'Away_Goals', ascending = False, inplace = True)

away_goals.head(1)

,team_long_name,Away_Goals
75,FC Barcelona,354


##### Answer: No, FC Barcelona scored the most points when playing away


<br>

#### 3. How many matches resulted in a tie?

In [75]:
query = """
WITH Ties AS (
    SELECT home_team_goal, away_team_goal
    FROM Match
    WHERE home_team_goal = away_team_goal
)

SELECT COUNT(*) AS Tie_Count
FROM Ties;
"""

tie_games = pd.read_sql_query(query, conn)

tie_games

,Tie_Count
0,6596


##### Answer: There were 6596 ties

<br>

#### 4. How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [151]:
query = """

    SELECT
    SUM(SUBSTR(player_name, -6) = ' Smith') AS smith_last_name
    , SUM(player_name LIKE '%SMITH%') AS smith_name_anywhere
    FROM Player


;
"""

smith = pd.read_sql_query(query, conn)

smith

,smith_last_name,smith_name_anywhere
0,15,18


##### Answer: 15 Players have the last name Smith, 18 Players have 'smith' anywhere in their name

<br>

#### 5. What was the median tie score? Use the value determined in the previous question for the number of tie games.

In [97]:
query = """
WITH Ties AS (
    SELECT home_team_goal, away_team_goal, PERCENT_RANK() OVER (ORDER BY home_team_goal) AS Rank
    FROM Match
    WHERE home_team_goal = away_team_goal   
)

SELECT home_team_goal, away_team_goal, Rank, (Rank - 0.5)  as Median
FROM Ties
ORDER BY Rank;
"""

tie_games = pd.read_sql_query(query, conn)

tie_games.head()
tie_games['Rank'].value_counts()

0.299924    3014
0.000000    1978
0.756937    1310
0.955572     264
0.995603      27
0.999697       2
1.000000       1
Name: Rank, dtype: int64

In [104]:
query = """
    SELECT home_team_goal, away_team_goal
    FROM Match
    WHERE home_team_goal = away_team_goal  
    
    
    
;
"""

tie_games = pd.read_sql_query(query, conn)

tie_games.describe()


,home_team_goal,away_team_goal
count,6596.000000,6596.000000
mean,0.993026,0.993026
std,0.839119,0.839119
min,0.000000,0.000000
25%,0.000000,0.000000
50%,1.000000,1.000000
75%,1.000000,1.000000
max,6.000000,6.000000


#### 6. What percentage of players prefer their left or right foot?

In [74]:
 Hint: Calculate either the 
# right or left foot, whichever is easier based on how you setup the problem.

query = """
SELECT 
CAST(SUM(preferred_foot = 'right') AS FLOAT)/ COUNT(*) AS Percent_Right
, (CAST(SUM(preferred_foot = 'left') AS FLOAT)/ COUNT(*)) AS Percent_Left
FROM Player_Attributes;
"""

right_left = pd.read_sql_query(query, conn)
right_left

,Percent_Right,Percent_Left
0,0.752313,0.243143


##### Answer: 75.23% of players prefer their right foot, 24.31% of players prefer their left foot. The rest didn't put a preference